# Example on the MNIST Digits Dataset

### 1. Import of the modules and the dataset

In [1]:
from neural_network import *
from keras.datasets import mnist

2023-08-30 20:34:33.546260: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 2. Load the MNIST dataset

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

### 3. Build the neural network

In [3]:
net = NeuralNetwork(
    Normalization(samples=X_train),
    Reshape(output_shape=(1, 28, 28)),
    Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2),
    BatchNorm2d(6),
    Tanh(),
    AvgPool2d(kernel_size=(2, 2), stride=(2, 2)),
    Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),
    BatchNorm2d(16),
    Tanh(),
    AvgPool2d(kernel_size=(2, 2), stride=(2, 2)),
    Flatten(),
    Linear(in_features=400, out_features=120),
    Tanh(),
    Linear(in_features=120, out_features=84),
    Tanh(),
    Linear(in_features=84, out_features=10),
    OutputLayer(activation_function="softmax", loss_function="categorical_cross_entropy")
)
print(net)

NeuralNetwork:
 (0) Normalization(norm=255.0, dtype=float32)
 (1) Reshape(output_shape=(1, 28, 28))
 (2) Conv2d(in_channels=1, out_channels=6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), initialization=xavier)
 (3) BatchNorm2d(6, eps=1e-05, momentum=0.1)
 (4) Tanh()
 (5) AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
 (6) Conv2d(in_channels=6, out_channels=16, kernel_size=(5, 5), stride=(1, 1), padding=(0, 0), optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), initialization=xavier)
 (7) BatchNorm2d(16, eps=1e-05, momentum=0.1)
 (8) Tanh()
 (9) AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
 (10) Flatten()
 (11) Linear(in_features=400, out_features=120, optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), initialization=xavier)
 (12) Tanh()
 (13) Linear(in_features=120, out_features=84, optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), ini

### 4. Choose the number of samples to train on

In [4]:
# Select N samples to train on:
N = 3000
X_train, y_train = X_train[:N], y_train[:N]
X_train.shape, y_train.shape

((3000, 28, 28), (3000,))

### 5. Train the neural network

In [5]:
net.fit(X_train, y_train, epochs=5, batch_size=64, shuffle=True)

Training on 3000 samples:
Epoch    1 of 5    	 Average Error = 0.730571 	 Average Accuracy = 80.70%                                   
Epoch    2 of 5    	 Average Error = 0.273386 	 Average Accuracy = 92.73%                                   
Epoch    3 of 5    	 Average Error = 0.181298 	 Average Accuracy = 95.20%                                   
Epoch    4 of 5    	 Average Error = 0.137516 	 Average Accuracy = 96.53%                                   
Epoch    5 of 5    	 Average Error = 0.110232 	 Average Accuracy = 97.13%                                   
Training time : 00 hours, 00 minutes, 38 seconds


### 6. Make predictions on the test samples

In [6]:
# Prediction on the 10,000 test samples:
y_pred = net.predict(X_test, to="labels")
# Shows the first 10 predicted labels vs the true labels:
y_pred[:10], y_test[:10]

(array([7, 2, 1, 0, 4, 1, 4, 9, 6, 9]),
 array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9], dtype=uint8))

### 7. Evaluate the accuracy score

In [7]:
print(f"Accuracy score on the test set: {accuracy_score(y_test, y_pred):.2%}")

Accuracy score on the test set: 95.10%


### 8. Build a confusion matrix

In [8]:
cm = confusion_matrix(y_test, y_pred)
print(f"Confusion matrix:\n{cm}")

Confusion matrix:
[[ 966    0    1    0    1    1    6    3    2    0]
 [   0 1121    4    4    0    1    3    0    1    1]
 [   6    1  993    5    9    0    4   10    4    0]
 [   0    0   20  965    1    3    0   10    2    9]
 [   1    5    2    0  947    0    5    1    1   20]
 [  12    3    6   42    3  788    8    6    7   17]
 [  15    2    2    1    9    2  924    0    2    1]
 [   0    5   34    2    1    0    0  980    1    5]
 [   9    4   29   13   10    0   13   13  865   18]
 [   3    3    1    5   14    1    1   18    2  961]]


### 9. Displays a classification report

In [9]:
print(classification_report(cm, formatted=True))

┌───────────────────┬───────────┬───────────┬───────────┬───────────┐
│       Class       │ Precision │   Recall  │  F1-Score │  Support  │
├───────────────────┼───────────┼───────────┼───────────┼───────────┤
│         0         │  95.45%   │  98.57%   │  96.99%   │    980    │
│         1         │  97.99%   │  98.77%   │  98.38%   │   1135    │
│         2         │  90.93%   │  96.22%   │  93.50%   │   1032    │
│         3         │  93.06%   │  95.54%   │  94.28%   │   1010    │
│         4         │  95.18%   │  96.44%   │  95.80%   │    982    │
│         5         │  98.99%   │  88.34%   │  93.36%   │    892    │
│         6         │  95.85%   │  96.45%   │  96.15%   │    958    │
│         7         │  94.14%   │  95.33%   │  94.73%   │   1028    │
│         8         │  97.52%   │  88.81%   │  92.96%   │    974    │
│         9         │  93.12%   │  95.24%   │  94.17%   │   1009    │
└───────────────────┴───────────┴───────────┴───────────┴───────────┘
